In [1]:
!pip install pandas numpy tensorflow scikit-learn

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
data = pd.read_csv("./XAUUSD_historical_data.csv")
data = data.iloc[::-1].reset_index(drop=True)
data["Target"] = data["Close"].shift(-1)
data = data.dropna()
data["Change(Pips)"] = data["Change(Pips)"]/100
X = data.drop(columns = ["Date", "Target"])
y = data["Target"]
data

,Date,Open,High,Low,Close,Change(Pips),Change(%),Target
0,2/12/2024 0:00,2025.25,2026.98,2011.30,2018.76,-6.49,-0.32,1991.28
1,2/13/2024 0:00,2018.76,2029.71,1990.19,1991.28,-27.48,-1.38,1993.75
2,2/14/2024 0:00,1991.24,1996.00,1985.27,1993.75,2.51,0.13,2003.26
3,2/15/2024 0:00,1993.79,2007.90,1990.18,2003.26,9.47,0.47,2013.01
4,2/16/2024 0:00,2003.35,2015.02,1995.22,2013.01,9.66,0.48,2016.11
...,...,...,...,...,...,...,...,...
179,10/13/2024 0:00,2650.29,2650.75,2647.63,2648.11,-2.18,-0.08,2649.51
180,10/14/2024 0:00,2645.21,2666.34,2643.32,2649.51,4.30,0.16,2660.42
181,10/15/2024 0:00,2649.30,2668.93,2638.23,2660.42,11.12,0.42,2673.90
182,10/16/2024 0:00,2659.97,2685.07,2658.78,2673.90,13.93,0.52,2693.55


In [4]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()

X = sc_X.fit_transform(np.array(X))
y = sc_y.fit_transform(np.array(y).reshape(-1,1))

X.shape

(184, 6)

In [5]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 1))

In [6]:
ann.compile(optimizer='adam', loss = 'mean_squared_error', metrics = ['mae'])

In [7]:
ann.fit(X, y, batch_size = 32, epochs = 500)

Epoch 1/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 1.1477 - mae: 0.7937
Epoch 2/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2307 - mae: 0.8277
Epoch 3/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1421 - mae: 0.7696
Epoch 4/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9517 - mae: 0.6718
Epoch 5/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8251 - mae: 0.6261
Epoch 6/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9526 - mae: 0.6648
Epoch 7/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7790 - mae: 0.5697 
Epoch 8/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7807 - mae: 0.5942
Epoch 9/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6888 - mae: 0.5508
Epoch 10/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7362 - mae: 0.5646
Epoch 11/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7467 - mae: 0.5683
Epoch 12/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7779 - mae: 0.5721
Epoch 13/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0

In [8]:
print(sc_y.inverse_transform(ann.predict(sc_X.transform([[2723.09,2740.60,2714.23,2719.67,-2.225,-0.08]])))[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
[2730.8918]


In [9]:
ann.save('./predictor_model.keras')